In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions


from scipy.optimize import minimize
from tqdm import tqdm
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Load Data

In [9]:
from sklearn.datasets import load_linnerud, load_iris, load_boston, load_breast_cancer, load_wine, load_digits

dataset = load_linnerud()
# dataset = load_boston()
# dataset = load_wine()
df = pd.DataFrame(dataset['data'])
target = dataset['target']

In [15]:
df = pd.read_csv('../data/abalone.csv', header=None)
df = df.drop(0, 1)
df = (df - df.mean())/(df.max() - df.min())
df0 = df.copy()
print(df.shape)
df.head()

(4177, 8)


,1,2,3,4,5,6,7,8
0,-0.093233,-0.072069,-0.039395,-0.111472,-0.090698,-0.104797,-0.088521,0.180940
1,-0.235124,-0.240137,-0.043820,-0.213650,-0.174760,-0.173922,-0.168242,-0.104774
2,0.008119,0.020368,-0.003997,-0.053743,-0.069178,-0.051473,-0.028730,-0.033346
3,-0.113503,-0.072069,-0.012846,-0.110764,-0.096750,-0.087681,-0.083538,0.002368
4,-0.262151,-0.256943,-0.052669,-0.220911,-0.181485,-0.185772,-0.183190,-0.104774


## Insert NaNs

In [ ]:
num_nan_cols = 3
nan_fraction = 0.4
nan_cols = np.random.random_integers(0, df.shape[1] - 1, num_nan_cols)
# print(df.isnull().mean())
for col in set(nan_cols):
    df.loc[df.sample(int(nan_fraction * len(df))).index, col] = np.nan
# print(df.isnull().mean())
# print(df.isnull().mean())

nan_coords = np.array(np.where(df.isnull().values)).T
print('Num nan places: {}'.format(nan_coords.shape[0]))

In [ ]:
df1 = df.loc[:, df.isnull().sum() == 0]
df2 = df.fillna(df.mean())
print(df1.shape, df2.shape)
arr_nan = df.values # с пропусками
arr_raw = df0.values # исходные
arr_known = df1.values # суженные до известных признаков
arr_pred = df2.values # текущие предсказанные 

In [ ]:
def iterative_process(update_function, max_iter=20, verbose=True):
    df_result = pd.DataFrame(np.zeros((max_iter, 2)), columns=['mae', 'rmse'])
    for i in range(max_iter):
        update_function()
        mae = functions.get_mae(arr_raw, arr_pred, nan_coords)
        rmse = functions.get_rmse(arr_raw, arr_pred, nan_coords)
        df_result.loc[i, 'mae'] = mae
        df_result.loc[i, 'rmse'] = rmse
        if verbose:
            print('\tIteration {}  mae {:.6} // rmse {:.6} '.format(i+1, mae, rmse))
    return df_result

# Local Approach

In [ ]:
from sklearn.neighbors import NearestNeighbors
k_local = 3
nbrs = NearestNeighbors(n_neighbors=k_local, algorithm='ball_tree').fit(arr_pred)

def get_delta_vec(teta):
    deltas = []
    _, heighbours = nbrs.kneighbors(arr_pred)
    for i, j in nan_coords:
        mean_neigh = arr_pred[heighbours[i]].mean(axis=0)[j]
        deltas.append(teta*(mean_neigh - arr_pred[i][j]))
    return np.array(deltas)

teta = 0.1
def update_local():
    deltas = get_delta_vec(teta)
    nbrs = NearestNeighbors(n_neighbors=k_local, algorithm='ball_tree').fit(arr_pred) # пересчитываем
    for j, (x,y) in enumerate(nan_coords):
        arr_pred[x, y] += deltas[j]

In [ ]:
dfr = iterative_process(update_local, max_iter=20, verbose=True)

# Optimization Approach

In [ ]:
def sq(a):
    return np.dot(a, a)

n = len(df0)

def get_known_table():
    ro_known_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(i, n):
            idx_nan_i = np.isnan(arr_nan[i])
            idx_nan_j = np.isnan(arr_nan[j])
            idx = ~idx_nan_i & ~idx_nan_j
            x_i = arr_nan[i, idx]
            x_j = arr_nan[j, idx]
            ro_known_table[i, j] = ro_known_table[j, i] = sq(x_i - x_j)
    return ro_known_table

def get_ro_1_unknown_table():
    ro_1_unknown_table = np.zeros((n, n))
    #это ro прибавить там где 1 компонента
    for i in tqdm(range(n)):
        for j in range(i, n):
            idx_nan_i = np.isnan(arr_nan[i])
            idx_nan_j = np.isnan(arr_nan[j])
            idx = (idx_nan_i & ~idx_nan_j) | (~idx_nan_i & idx_nan_j)
            x_i = arr_pred[i, idx]
            x_j = arr_pred[j, idx]
            ro_1_unknown_table[i, j] = ro_1_unknown_table[j, i] = sq(x_i - x_j)
    return ro_1_unknown_table

# по всем признакам
def get_ro_2_unknown_table():
    '''
    при первом запуске обычно 0, так как заполняю пропуски ОДНИМ И ТЕМ ЖЕ СРЕДНИМ ЗНАЧЕНИЕМ
    '''
    ro_2_unknown_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(i+1, n):
            idx_nan_i = np.isnan(arr_nan[i])
            idx_nan_j = np.isnan(arr_nan[j])
            idx = (idx_nan_i & idx_nan_j)
            x_i = arr_pred[i, idx]
            x_j = arr_pred[j, idx]
            ro_2_unknown_table[i, j] = ro_2_unknown_table[j, i] = sq(x_i - x_j)
    return ro_2_unknown_table

In [ ]:
ro_known_table = get_known_table()
ro_1_unknown_table = get_ro_1_unknown_table()
ro_2_unknown_table = get_ro_2_unknown_table() # здесь какого-то хера одни нули!!!

In [ ]:
def F_functional(k_arr, ro_1_unknown_table, ro_2_unknown_table, ftype='f1'):
    f = 0
    for i in tqdm(range(n)):
        for j in range(i+1, n):
            ro1 = ro_known_table[i, j] # известно оба 
            ro2 = ro_1_unknown_table[i, j] # известен 1 
            ro3 = ro_2_unknown_table[i, j] # не известно ниче
            
            r1 = np.sqrt(ro1 + ro2 + ro3) #по всем признакам(р)
            if ftype == 'f1':
                r2 = np.sqrt(ro1) # по известным признакам(р+)
            else:
                r2 = np.sqrt(ro1 + ro2)
            f += (r1 - k_arr[i, j]*r2)**2
    return f

In [ ]:
### ПРОВЕРИТЬ ГРАДИЕНТ НА 0 ДЛЯ Ф2

def grad_component(i, j, k_arr, ftype='f1'):
    s = 0
    for i1 in range(n):
        ro1 = ro_known_table[i, i1]
        if ro1 <= 0:
            continue
        if i1 == i:
            continue
        ro2 = ro_1_unknown_table[i, i1]
        ro3 = ro_2_unknown_table[i, i1]
        
        if ftype == 'f2':
            r1 = np.sqrt(ro1 + ro2 + ro3) #по всем признакам(р)
            r2 = np.sqrt(ro1 + ro2) # по известным признакам(р+)
            delta_x1 = (arr_pred[i, j] - arr_pred[i1, j])
            if not np.isnan(arr_nan[i1, j]):
                delta_x2 = delta_x1
            else:
                delta_x2 = 0
            delta_s = 2*(r1 - k_arr[i1, i]*r2)*(delta_x1/r1 - k_arr[i1, i]*delta_x2/r2)
#             print(1 - k_arr[i1, i]*r2/r1)
            s += delta_s
        else:
            r1 = np.sqrt(ro1 + ro2 + ro3) #по всем признакам(р)
            r2 = np.sqrt(ro1) # по известным признакам(р+)
            s += 2 * (1 - k_arr[i1, i]*r2/r1) * (arr_pred[i, j] - arr_pred[i1, j])
    return s

def get_full_grad(nan_coords, k_arr, ro_1_unknown_table, ro_2_unknown_table, ftype='f1'):
    grad = []
    for i, j in nan_coords:
        grad.append(grad_component(i, j, k_arr, ftype))
    return np.array(grad)

In [ ]:
k_arr = np.ones((n, n))
ftype = 'f1'
c = F_functional(k_arr, ro_1_unknown_table, ro_2_unknown_table, ftype)

In [ ]:
# Для F2 нужно делать не единички, а нормировать
# k_arr[i, j] = n - n2 / n (то есть все признаки без двух пропусков/на все признаки)
k_arr = np.ones((n, n))
alpha = 2
for i in tqdm(range(n)):
    for j in range(i+1, n):
        idx_nan_i = np.isnan(arr_nan[i])
        idx_nan_j = np.isnan(arr_nan[j])
        idx = (idx_nan_i & idx_nan_j)
        k_arr[i, j] = k_arr[j, i] = (n - idx.sum()) / n
ftype = 'f2'
c = F_functional(k_arr, ro_1_unknown_table, ro_2_unknown_table, ftype)

In [ ]:
grad = get_full_grad(nan_coords, k_arr, ro_1_unknown_table, ro_2_unknown_table, ftype)

In [ ]:
c_array = []

ro1t = []
ro2t = []
def update_opt():
    global ro_1_unknown_table, ro_2_unknown_table
    grad = get_full_grad(nan_coords, k_arr, ro_1_unknown_table, ro_2_unknown_table, ftype)
    for j, (x,y) in enumerate(nan_coords):
        arr_pred[x, y] = arr_pred[x, y] - alpha*grad[j]
        
    ro_1_unknown_table = get_ro_1_unknown_table()
    ro_2_unknown_table = get_ro_2_unknown_table()
    c = F_functional(k_arr, ro_1_unknown_table, ro_2_unknown_table, ftype)
    c_array.append(c)
    ro1t.append(ro_1_unknown_table)
    ro2t.append(ro_2_unknown_table)

In [ ]:
c_array, ftype


In [ ]:
for i in range(7):
    print((ro2t[i] - ro2t[i+1]).max())

In [ ]:
max_iter = 30

dfr = iterative_process(update_opt, max_iter=20, verbose=True)

# SNE Approach

In [ ]:
n = len(df0)
# таблица старых значений
Exp1_table = np.zeros((n, n))
for i in tqdm(range(n)):
    for j in range(n):
        Exp1_table[i][j] = np.exp(-norm(arr_known[i] - arr_known[j])**2)
        
P1_table = np.zeros((n, n))
for i in tqdm(range(n)):
    for j in range(n):
        a = Exp1_table[j][i]
        b = Exp1_table[i].sum() - 1 # 1 = Exp1_table[i][i]
        P1_table[j][i] = a / b

In [ ]:
def get_exp2_table():
    Exp2_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(n):
            Exp2_table[i][j] = np.exp(-norm(arr_pred[i] - arr_pred[j])**2)
    return Exp2_table

def get_p2_table():
    Exp2_table = get_exp2_table()
    P2_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(n):
            a = Exp2_table[j][i]
            b = Exp2_table[i].sum() - 1 # 1 = Exp1_table[i][i]
            P2_table[j][i] = a / b
    return P2_table
P2_table = get_p2_table()

In [ ]:
def KL():
    s = 0
    for i in range(n):
        for j in range(n):
            s += P1_table[j][i] * np.log(P1_table[j][i] / P2_table[j][i])
    return s

def get_grad_sne(i1, i2):
    def get_i_part(i):
        d = 2*(arr_pred[i1][i2] - arr_pred[i][i2])
        s = (P1_table[i1][i]+P1_table[i][i1]) - \
            P2_table[i1][i]*(1+P1_table[i][i]) - \
            P2_table[i][i1]*(1+P1_table[i1][i1])
        return s*d
    return sum(get_i_part(i) for i in range(n) if i!=i1)

def get_full_grad_sne(nan_coords):
    n_gaps = len(nan_coords)
    grad = np.zeros(n_gaps)
    for i in range(n_gaps):
        i1, i2 = nan_coords[i]
        grad[i] = get_grad_sne(i1, i2)
    return grad

In [ ]:
def update_sne():
    grad = get_full_grad_sne(nan_coords)
    for j, (x,y) in enumerate(nan_coords):
        arr_pred[x, y] = arr_pred[x, y] - alpha*grad[j]
    P2_table = get_p2_table()
    c = KL()
    c_array.append(c)

In [ ]:
alpha = 0.15
c_array = []
dfr = iterative_process(update_sne, max_iter=150, verbose=True)

# AVO APPROACH
Единственный подход, который не содержит оптимизайионный подход

In [ ]:
def Cnk(n, k):
    a = b = c = tmp = 1
    for i in range(1, n+1):
        tmp *= i
        if i == n-k:
            a = tmp
        if i == k:
            b = tmp
        if i == n:
            c = tmp
    return c / (a*b)

In [ ]:
def get_epsilons(data):
    def get_e(a):
        return np.abs(a - a[:, None]).mean()
    return np.array([get_e(feat) for feat in data.T])

In [ ]:
def get_gamma(data, x, k, epsilons=None):
    '''
    необходимо вернуть кол-во пар values которые a<x<b
    a1 ... a_i < x < a_i+1 ... an
    '''
    if epsilons is not None:
        Gamma = 0
        for x_i in data:
            d = (np.abs(x_i - x) < epsilons).sum()
            Gamma += Cnk(d, k)
        return Gamma
    # расстояния между парами объектов   
    n = data.shape[0]
    n_feat = data.shape[1]
    maps = np.zeros(data.T.shape)
    for i in range(n_feat):
        maps[i] = data.T[i] <= x[i] 
    maps = maps.T
    Gamma = 0
    for i in tqdm(range(n)):
        for j in range(i, n):
            d = (maps[i] + maps[j] == 1).sum()
            Gamma += Cnk(d, k)
    return Gamma*2/(n*(n-1))

In [ ]:
def predict(data, y, x, k, epsilons=None, scores=False):
    n_class = max(y) + 1
    g_classes = np.zeros(n_class)
    for i in range(n_class):
        c_data = data[y == i]
        g_classes[i] = get_gamma(c_data, x, k, epsilons)
    if scores:
        return g_classes
    return np.argmax(g_classes)

def predict_vect(X_train, y_train, X_test, k, epsilons=None, scores=False):
    return np.array([predict(X_train, y_train, x, k, epsilons, scores) for x in X_test])

In [ ]:
# итеративный процесс
def solve_avo(df, x, y):
    x_mask = df.iloc[:, y].isnull()
    y_mask = df.isnull().sum() > 0
    y_mask = y_mask[~y_mask].index

    X_train = df.iloc[x_mask[~x_mask].index, y_mask]
    X_test = df.iloc[x, y_mask].values.reshape(1, -1)

    y_train_raw = df.iloc[x_mask[~x_mask].index, y]
    y_train = np.argsort(y_train_raw)
    
    eps = get_epsilons(X_train.values)
    gammas_ki = predict(X_train.values, y_train, X_test, 3, eps, True)
    sizes_ki = y_train.value_counts().sort_index().values
    full_range = np.array(range(sizes_ki.size))
    sort_index = solve_avo_gap(gammas_ki, sizes_ki, full_range)
    return y_train_raw.sort_values().iloc[sort_index]
    
def solve_avo_gap(gammas_ki, sizes_ki, full_range):
#     print(full_range)
    if len(full_range) == 1:
        return full_range[0]
    left_range = full_range[:full_range.size//2]
    right_range = full_range[full_range.size//2:]
    left_score = get_gamma_range(gammas_ki, sizes_ki, left_range)
    right_score = get_gamma_range(gammas_ki, sizes_ki, right_range)
    if left_score < right_score:
        return solve_avo_gap(gammas_ki, sizes_ki, right_range)
    else:
        return solve_avo_gap(gammas_ki, sizes_ki, left_range)
        
def get_gamma_range(gammas_ki, sizes_ki, y_range):
    sum_gammas = gammas_ki[y_range].sum()
    sum_sizes = sizes_ki[y_range].sum()
    if sum_sizes == 0:
        return 0
    return sum_gammas/sum_sizes

In [ ]:
# по всем пропускам из таблицы запускаем и проходим
for x, y in tqdm(nan_coords):
    arr_pred[x, y] = solve_avo(df, x, y)

In [ ]:
mae = functions.get_mae(arr_raw, arr_pred, nan_coords)
rmse = functions.get_rmse(arr_raw, arr_pred, nan_coords)
print(mae, rmse)

In [ ]:
arr_pred